# Introduction to Recommendation Systems

In this tutorial we are going to use a [deep autoencoder](https://arxiv.org/abs/1708.01715) to perform collaborative filtering in the [Netflix dataset](https://netflixprize.com/). 

[Collaborative filtering](https://en.wikipedia.org/wiki/Collaborative_filtering) is one of the most pupular techniques in recommendation systems. It is based on inferring the missing entries in an `mxn` matrix, `R`, whose `(i, j)` entry describes the ratings given by the `ith` user to the `jth` item. The performance is then measured using Root
Mean Squared Error (RMSE).

<p align="center">
    <img src="https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif" width=300px/>
</p>

The code in this tutorial is done with [PyTorch](http://pytorch.org/) and is based on [this repo](https://github.com/NVIDIA/DeepRecommender) by NVIDIA.

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import torch
from utils import get_gpu_name, get_number_processors, get_gpu_memory, get_cuda_version

print("OS: ", sys.platform)
print("Python: ", sys.version)
print("PyTorch: ", torch.__version__)
print("Numpy: ", np.__version__)
print("Number of CPU processors: ", get_number_processors())
print("GPU: ", get_gpu_name())
print("GPU memory: ", get_gpu_memory())
print("CUDA: ", get_cuda_version())

%matplotlib inline
%load_ext autoreload
%autoreload 2

OS:  linux
Python:  3.5.4 | packaged by conda-forge | (default, Nov  4 2017, 10:11:29) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
PyTorch:  0.3.0.post4
Numpy:  1.14.0
Number of CPU processors:  24
GPU:  ['Tesla M60', 'Tesla M60', 'Tesla M60', 'Tesla M60']
GPU memory:  ['8123 MiB', '8123 MiB', '8123 MiB', '8123 MiB']
CUDA:  CUDA Version 8.0.61


## Dataset: Netflix

This dataset was constructed to support participants in the [Netflix Prize](http://www.netflixprize.com). The movie rating files contain over 100 million ratings from 480 thousand randomly-chosen, anonymous Netflix customers over 17 thousand movie titles.  The data were collected between October, 1998 and December, 2005 and reflect the distribution of all ratings received during this period.  The ratings are on a scale from 1 to 5 (integral) stars.

The dataset can be [downloaded here](http://academictorrents.com/details/9b13183dc4d60676b773c9e2cd6de5e5542cee9a). To uncompress it:

```bash
tar -xvf nf_prize_dataset.tar.gz
tar -xf download/training_set.tar
```

When we download the data, there are two important files:

1) The file `training_set.tar` is a tar of a directory containing 17770 files, one per movie.  The first line of each file contains the movie id followed by a colon.  Each subsequent line in the file corresponds to a rating from a customer and its date in the following format:

CustomerID,Rating,Date
- MovieIDs range from 1 to 17770 sequentially.
- CustomerIDs range from 1 to 2649429, with gaps. There are 480189 users.
- Ratings are on a five star (integral) scale from 1 to 5.
- Dates have the format YYYY-MM-DD.

2) Movie information in `movie_titles.txt` is in the following format:

MovieID,YearOfRelease,Title

- MovieID do not correspond to actual Netflix movie ids or IMDB movie ids.
- YearOfRelease can range from 1890 to 2005 and may correspond to the release of corresponding DVD, not necessarily its theaterical release.
- Title in English is the Netflix movie.

### Data prep

The first step is to covert the data to the correct format for the autoencoder to read. For that we execute the following:

In [2]:
DATA_ROOT = '/datadrive'
NF_PRIZE_DATASET = os.path.join(DATA_ROOT, 'netflix','download','training_set') #location of extracted data
NF_DATA = 'Netflix'

In [ ]:
%%time
%run ./DeepRecommender/data_utils/netflix_data_convert.py $NF_PRIZE_DATASET $NF_DATA

Processing: /datadrive/netflix/download/training_set/mv_0012918.txt
Processing: /datadrive/netflix/download/training_set/mv_0005006.txt
Processing: /datadrive/netflix/download/training_set/mv_0016268.txt
Processing: /datadrive/netflix/download/training_set/mv_0015225.txt
Processing: /datadrive/netflix/download/training_set/mv_0003936.txt
Processing: /datadrive/netflix/download/training_set/mv_0013837.txt
Processing: /datadrive/netflix/download/training_set/mv_0007131.txt
Processing: /datadrive/netflix/download/training_set/mv_0012557.txt
Processing: /datadrive/netflix/download/training_set/mv_0002514.txt
Processing: /datadrive/netflix/download/training_set/mv_0007157.txt
Processing: /datadrive/netflix/download/training_set/mv_0013581.txt
Processing: /datadrive/netflix/download/training_set/mv_0000303.txt
Processing: /datadrive/netflix/download/training_set/mv_0001795.txt
Processing: /datadrive/netflix/download/training_set/mv_0009954.txt
Processing: /datadrive/netflix/download/training

Processing: /datadrive/netflix/download/training_set/mv_0000342.txt
Processing: /datadrive/netflix/download/training_set/mv_0013785.txt
Processing: /datadrive/netflix/download/training_set/mv_0016286.txt
Processing: /datadrive/netflix/download/training_set/mv_0008342.txt
Processing: /datadrive/netflix/download/training_set/mv_0008188.txt
Processing: /datadrive/netflix/download/training_set/mv_0007768.txt
Processing: /datadrive/netflix/download/training_set/mv_0003230.txt
Processing: /datadrive/netflix/download/training_set/mv_0015491.txt
Processing: /datadrive/netflix/download/training_set/mv_0007692.txt
Processing: /datadrive/netflix/download/training_set/mv_0016502.txt
Processing: /datadrive/netflix/download/training_set/mv_0002381.txt
Processing: /datadrive/netflix/download/training_set/mv_0013316.txt
Processing: /datadrive/netflix/download/training_set/mv_0002610.txt
Processing: /datadrive/netflix/download/training_set/mv_0001234.txt
Processing: /datadrive/netflix/download/training

Processing: /datadrive/netflix/download/training_set/mv_0007594.txt
Processing: /datadrive/netflix/download/training_set/mv_0005957.txt
Processing: /datadrive/netflix/download/training_set/mv_0004270.txt
Processing: /datadrive/netflix/download/training_set/mv_0015336.txt
Processing: /datadrive/netflix/download/training_set/mv_0015849.txt
Processing: /datadrive/netflix/download/training_set/mv_0002029.txt
Processing: /datadrive/netflix/download/training_set/mv_0016259.txt
Processing: /datadrive/netflix/download/training_set/mv_0016459.txt
Processing: /datadrive/netflix/download/training_set/mv_0010295.txt
Processing: /datadrive/netflix/download/training_set/mv_0016912.txt
Processing: /datadrive/netflix/download/training_set/mv_0002878.txt
Processing: /datadrive/netflix/download/training_set/mv_0000841.txt
Processing: /datadrive/netflix/download/training_set/mv_0005850.txt
Processing: /datadrive/netflix/download/training_set/mv_0009982.txt
Processing: /datadrive/netflix/download/training

Processing: /datadrive/netflix/download/training_set/mv_0014135.txt
Processing: /datadrive/netflix/download/training_set/mv_0014159.txt
Processing: /datadrive/netflix/download/training_set/mv_0000783.txt
Processing: /datadrive/netflix/download/training_set/mv_0017094.txt
Processing: /datadrive/netflix/download/training_set/mv_0011765.txt
Processing: /datadrive/netflix/download/training_set/mv_0014617.txt
Processing: /datadrive/netflix/download/training_set/mv_0003283.txt
Processing: /datadrive/netflix/download/training_set/mv_0001952.txt
Processing: /datadrive/netflix/download/training_set/mv_0005115.txt
Processing: /datadrive/netflix/download/training_set/mv_0016103.txt
Processing: /datadrive/netflix/download/training_set/mv_0015032.txt
Processing: /datadrive/netflix/download/training_set/mv_0000251.txt
Processing: /datadrive/netflix/download/training_set/mv_0015936.txt
Processing: /datadrive/netflix/download/training_set/mv_0002627.txt
Processing: /datadrive/netflix/download/training

Processing: /datadrive/netflix/download/training_set/mv_0004191.txt
Processing: /datadrive/netflix/download/training_set/mv_0000781.txt
Processing: /datadrive/netflix/download/training_set/mv_0013570.txt
Processing: /datadrive/netflix/download/training_set/mv_0000786.txt
Processing: /datadrive/netflix/download/training_set/mv_0013275.txt
Processing: /datadrive/netflix/download/training_set/mv_0011103.txt
Processing: /datadrive/netflix/download/training_set/mv_0001452.txt
Processing: /datadrive/netflix/download/training_set/mv_0015724.txt
Processing: /datadrive/netflix/download/training_set/mv_0004859.txt
Processing: /datadrive/netflix/download/training_set/mv_0010860.txt
Processing: /datadrive/netflix/download/training_set/mv_0007853.txt
Processing: /datadrive/netflix/download/training_set/mv_0002712.txt
Processing: /datadrive/netflix/download/training_set/mv_0006081.txt
Processing: /datadrive/netflix/download/training_set/mv_0009914.txt
Processing: /datadrive/netflix/download/training

...

Processing: /datadrive/netflix/download/training_set/mv_0000103.txt
Processing: /datadrive/netflix/download/training_set/mv_0005152.txt
Processing: /datadrive/netflix/download/training_set/mv_0016296.txt
Processing: /datadrive/netflix/download/training_set/mv_0015956.txt
Processing: /datadrive/netflix/download/training_set/mv_0001588.txt
Processing: /datadrive/netflix/download/training_set/mv_0003439.txt
Processing: /datadrive/netflix/download/training_set/mv_0008926.txt
Processing: /datadrive/netflix/download/training_set/mv_0013196.txt
Processing: /datadrive/netflix/download/training_set/mv_0000195.txt
Processing: /datadrive/netflix/download/training_set/mv_0001187.txt
Processing: /datadrive/netflix/download/training_set/mv_0016783.txt
Processing: /datadrive/netflix/download/training_set/mv_0006132.txt
Processing: /datadrive/netflix/download/training_set/mv_0006955.txt
Processing: /datadrive/netflix/download/training_set/mv_0001642.txt
Processing: /datadrive/netflix/download/training

Processing: /datadrive/netflix/download/training_set/mv_0009613.txt
Processing: /datadrive/netflix/download/training_set/mv_0015269.txt
Processing: /datadrive/netflix/download/training_set/mv_0007311.txt
Processing: /datadrive/netflix/download/training_set/mv_0014837.txt
Processing: /datadrive/netflix/download/training_set/mv_0015569.txt
Processing: /datadrive/netflix/download/training_set/mv_0012246.txt
Processing: /datadrive/netflix/download/training_set/mv_0008433.txt
Processing: /datadrive/netflix/download/training_set/mv_0005358.txt
Processing: /datadrive/netflix/download/training_set/mv_0010284.txt
Processing: /datadrive/netflix/download/training_set/mv_0017268.txt
Processing: /datadrive/netflix/download/training_set/mv_0010090.txt
Processing: /datadrive/netflix/download/training_set/mv_0009184.txt
Processing: /datadrive/netflix/download/training_set/mv_0009044.txt
Processing: /datadrive/netflix/download/training_set/mv_0001260.txt
Processing: /datadrive/netflix/download/training

Processing: /datadrive/netflix/download/training_set/mv_0014173.txt
Processing: /datadrive/netflix/download/training_set/mv_0006791.txt
Processing: /datadrive/netflix/download/training_set/mv_0011090.txt
Processing: /datadrive/netflix/download/training_set/mv_0011297.txt
Processing: /datadrive/netflix/download/training_set/mv_0009441.txt
Processing: /datadrive/netflix/download/training_set/mv_0006256.txt
Processing: /datadrive/netflix/download/training_set/mv_0010822.txt
Processing: /datadrive/netflix/download/training_set/mv_0009826.txt
Processing: /datadrive/netflix/download/training_set/mv_0005105.txt
Processing: /datadrive/netflix/download/training_set/mv_0008717.txt
Processing: /datadrive/netflix/download/training_set/mv_0015745.txt
Processing: /datadrive/netflix/download/training_set/mv_0011810.txt
Processing: /datadrive/netflix/download/training_set/mv_0007651.txt
Processing: /datadrive/netflix/download/training_set/mv_0011048.txt
Processing: /datadrive/netflix/download/training

Processing: /datadrive/netflix/download/training_set/mv_0005970.txt
Processing: /datadrive/netflix/download/training_set/mv_0001372.txt
Processing: /datadrive/netflix/download/training_set/mv_0015400.txt
Processing: /datadrive/netflix/download/training_set/mv_0000188.txt
Processing: /datadrive/netflix/download/training_set/mv_0006003.txt
Processing: /datadrive/netflix/download/training_set/mv_0005815.txt
Processing: /datadrive/netflix/download/training_set/mv_0014340.txt
Processing: /datadrive/netflix/download/training_set/mv_0016990.txt
Processing: /datadrive/netflix/download/training_set/mv_0014128.txt
Processing: /datadrive/netflix/download/training_set/mv_0017245.txt
Processing: /datadrive/netflix/download/training_set/mv_0004756.txt
Processing: /datadrive/netflix/download/training_set/mv_0006802.txt
Processing: /datadrive/netflix/download/training_set/mv_0000980.txt
Processing: /datadrive/netflix/download/training_set/mv_0016104.txt
Processing: /datadrive/netflix/download/training

Processing: /datadrive/netflix/download/training_set/mv_0014363.txt
Processing: /datadrive/netflix/download/training_set/mv_0006917.txt
Processing: /datadrive/netflix/download/training_set/mv_0002565.txt
Processing: /datadrive/netflix/download/training_set/mv_0012318.txt
Processing: /datadrive/netflix/download/training_set/mv_0005353.txt
Processing: /datadrive/netflix/download/training_set/mv_0015212.txt
Processing: /datadrive/netflix/download/training_set/mv_0002675.txt
Processing: /datadrive/netflix/download/training_set/mv_0010882.txt
Processing: /datadrive/netflix/download/training_set/mv_0013616.txt
Processing: /datadrive/netflix/download/training_set/mv_0017452.txt
Processing: /datadrive/netflix/download/training_set/mv_0017071.txt
Processing: /datadrive/netflix/download/training_set/mv_0004152.txt
Processing: /datadrive/netflix/download/training_set/mv_0009174.txt
Processing: /datadrive/netflix/download/training_set/mv_0004138.txt
Processing: /datadrive/netflix/download/training

The script splitted the data into train, test and validation set, creating files with three columns: `CustomerID,MovieID,Rating`. The data is splitted over time generating 4 datasets: Netflix 3months, Netflix 6 months, Netflix 1 year and Netflix full. Here there is a table with some details of each dataset:

| Dataset  | Netflix 3 months | Netflix 6 months | Netflix 1 year | Netflix full |
| -------- | ---------------- | ---------------- | ----------- |  ------------ |
| Ratings train | 13,675,402 | 29,179,009 | 41,451,832 | 98,074,901 |
| Users train | 311,315 |390,795  | 345,855 | 477,412 |
| Items train | 17,736 |17,757  | 16,907 | 17,768 |
| Time range train | 2005-09-01 to 2005-11-31 | 2005-06-01 to 2005-11-31 | 2004-06-01 to 2005-05-31 | 1999-12-01 to 2005-11-31
|  |  |  |   | |
| Ratings test | 2,082,559 | 2,175,535  | 3,888,684| 2,250,481 |
| Users test | 160,906 | 169,541  | 197,951| 173,482 |
| Items test | 17,261 | 17,290  | 16,506| 17,305 |
| Time range test | 2005-12-01 to 2005-12-31 | 2005-12-01 to 2005-12-31 | 2005-06-01 to 2005-06-31 | 2005-12-01 to 2005-12-31

Let's take a look at one of the files.

In [ ]:
nf_3m_valid = os.path.join(NF_DATA, 'N3M_VALID', 'n3m.valid.txt')
nf_3m_valid = '/home/hoaphumanoid/notebooks/repos/DeepRecommender/Netflix/N3M_VALID/n3m.valid.txt'
df = pd.read_csv(nf_3m_valid, colums=['CustomerID','MovieID','Rating'])
print(df.shape)
df.head()

## Deep Autoencoder for Collaborative Filtering

Once we have the data, let's explain in some detail the model that we are going to use. The [model](https://arxiv.org/abs/1708.01715) developed by NVIDIA folks is a Deep autoencoder with 6 layers with non-linear activation function SELU (scaled exponential linear units), dropout and iterative dense refeeding.



